In [12]:
#device_reset!()
include("vgg.jl")
include("layers.jl")
include("helpers.jl")
include("dataset.jl")
include("ssd4.jl")
include("utilities.jl")
include("train.jl")
include("transformations.jl")
include("config.jl")
using Dates


In [13]:
batch_size = 32
trainval_VOC2007_dataset = PascalVOC(trainval_VOC2007, annotation_path_trainval_VOC2007,
    images_path_trainval_VOC2007,"TRAIN",dtype = atype, batchsize =batch_size)

test_VOC2007_dateset = PascalVOC(test_VOC2007, annotation_path_test_VOC2007, images_path_test_VOC2007,
    "TEST", dtype=atype,batchsize=batch_size)

PascalVOC(["000001", "000002", "000003", "000004", "000006", "000008", "000010", "000011", "000013", "000014"  …  "009943", "009948", "009951", "009952", "009953", "009956", "009957", "009960", "009962", "009963"], 32, false, 4952, true, "TEST", "/kuacc/users/ckoksal20/VOC2007Test/VOCdevkit/VOC2007/Annotations", "/kuacc/users/ckoksal20/VOC2007Test/VOCdevkit/VOC2007/JPEGImages", KnetArray{Float32,N} where N)


train_VOC2012_dataset = PascalVOC(train_VOC2012, annotation_path_train_VOC2012
    , images_path_train_VOC2012,"TRAIN",dtype = atype, batchsize =batch_size)

val_VOC2012_dataset = PascalVOC(validation_VOC2012, annotation_path_train_VOC2012
    , images_path_train_VOC2012,"TEST",dtype = atype, batchsize =batch_size)

#testset does not have annotation in cluster
test_VOC2012_dataset = PascalVOC(test_VOC2012,annotation_path_test_VOC2012,
    images_path_test_VOC2012, "TEST", dtype=atype,batchsize=batch_size)



lrdecay! (generic function with 2 methods)

In [15]:
function initopt!(model::SSD300, startLr = 1e-3,gclip = 0)
    for par in params(model)
        # Authors initialzied learning rates of bias 2x of weights learning rate
        if size(par)[1:2] == (1,1) && size(par)[end] == 1 #bias
            par.opt = Momentum(;lr=2*startLr, gclip=0, gamma = 0.9)
        else
            par.opt = Momentum(;lr=startLr, gclip=0,gamma = 0.9)
        end
    end
end
   
function currentTime()
    dt = now()
    return Dates.format(dt, "yyyy-mm-dd_HH:MM:SS")
end
function lrdecay!(model::SSD300, decay = 0.1)
    for param in params(model)
        param.opt.lr = param.opt.lr*decay
    end
end



initopt! (generic function with 3 methods)

currentTime (generic function with 1 method)

In [17]:
ssd300 = SSD300("CLUSTER");
initopt!(ssd300)

In [7]:
#(x,bounding_boxes,labels) = first(train_VOC2012_dataset);

In [8]:
#ssd300(x,bounding_boxes,labels) 

#Overfit on single example
for i in 1:10000
    println("Iteration : ",i)
   momentum!(ssd300,[(x,bounding_boxes,labels)])
end

#Overfit on single example
for i in 1:10000
    J = @diff ssd300(x,bounding_boxes,labels)
    for par in params(ssd300)
        g = grad(J, par)
        Knet.update!(Knet.value(par), g, par.opt)
    end
    
   # momentum!(ssd300,[(x,bounding_boxes,labels)],lr = 1e-3, gamma=0.9)
end

In [18]:
iterations = 30000
datasets = [trainval_VOC2007_dataset]

total_num_batch = 0
for data in datasets
    total_num_batch += length(data)
end
    
epochs = floor(iterations/ total_num_batch)


382.0

for (x,bounding_boxes,labels) in train_dataset
    #sgd!(ssd300,[(x,bounding_boxes,labels)],lr = 1e-3)
    println(siz(x))
end
    

for (x,bounding_boxes,labels) in train_dataset
    println(typeof(x))
    ssd300(x,bounding_boxes,labels)
    break
end

iterations = 120000  # number of iterations to train
#workers = 4  # number of workers for loading data in the DataLoader
#print_freq = 200  # print training status every __ batches
lr = 1e-3  # learning rate
decay_lr_at = [80000, 100000]  # decay learning rate after these many iterations
decay_lr_to = 0.1  # decay learning rate to this fraction of the existing learning rate
momentum = 0.9  # momentum
weight_decay = 5e-4  # weight decay
epochs = iterations / (length(train_dataset) / 32)
decay_lr_at = [it / (length(train_dataset) / 32) for it in decay_lr_at]
decay_lr_at = floor.(decay_lr_at)

function writeToLogFile(filename , yazi)
    open("/kuacc/users/ckoksal20/log_files/$filename", "a") do f
    write(f, "$yazi\n")
    end
end
     

In [ ]:
epochs

In [19]:
function mytrain!(model::SSD300, train_datasets, test_data::PascalVOC,
                  period::Int=2000, iters::Int=50000;learning_rate = 1e-3 , learning_rate_decay = 0.1 )
    
    train_loss = []
    test_loss = []
    iteration = 0
    curTime = currentTime()
    tr_loss = 0
    tst_loss = 0
    
    filename="log_ssd300_2012_VOC2007trainval_normal_l1_batch_$batch_size-iteration_$iterations-$curTime.txt"
    for epoch in 1:epochs
        start_epoch_time = now()
        for train_data in train_datasets
            for (x,bounding_boxes,labels) in train_data
                if iteration == 20000|| (iteration > 20000 && iteration % 10000 == 0)
                    
                    lrdecay!(model,learning_rate_decay)
                    
                    println("Learning_rate decay is applied")
                    open("/kuacc/users/ckoksal20/log_files/$filename", "a") do f
                        write(f, "Learning rate changed to $learning_rate epoch $epoch ")

                    end
                    curTime = currentTime()
                    Knet.save("/kuacc/users/ckoksal20/trained_models/model_ssd30_normal_l1_VOC2007trainval_curiteration-$iteration-$curTime.jld2","model_$iteration",model)

                end

                if iteration%period == 0
                    tr_loss = 0
                    for train_data in train_datasets
                        tr_loss+= model(train_data) 
                    end
                    tr_loss = tr_loss/ length(train_datasets)
                        
                    tst_loss = model(test_data)
                    println("Iteration", iteration, "training loss : ",tr_loss, "test_loss : ",tst_loss)
                    open("/kuacc/users/ckoksal20/log_files/$filename", "a") do f
                        write(f, "Iteration: $iteration, training loss $tr_loss, test_loss $tst_loss \n")
                    end
                    append!(train_loss,tr_loss )
                    append!(test_loss,tst_loss )
                end
                sgd!(ssd300,[(x,bounding_boxes,labels)],lr = learning_rate)
                iteration +=1
            end
        end
    end_epoch_time = now()
    elapsed_time = ((end_epoch_time -start_epoch_time).value)/1000/60
        
    println("Elapsed time $elapsed_time min epoch : $epoch\n")    
    open("/kuacc/users/ckoksal20/log_files/$filename", "a") do f
        write(f, "Elapsed time $elapsed_time min epoch : $epoch\n")
        write(f,"Iteration: $iteration, training loss : $tr_loss, test_loss : $tst_loss \n")
    
        end
    end
        
    return 0:period:iteration, train_loss, test_loss
end



mytrain! (generic function with 3 methods)

In [ ]:
iters, trnloss, tstloss = mytrain!(ssd300,datasets,test_VOC2007_dateset;learning_rate = 1e-3 )

Total loss : 25.422195 Localization Loss : 4.5341387 Confidence Loss :20.888058
Total loss : 24.564125 Localization Loss : 4.553466 Confidence Loss :20.010658
Total loss : 24.221725 Localization Loss : 4.7184978 Confidence Loss :19.503227
Total loss : 24.055332 Localization Loss : 4.982872 Confidence Loss :19.07246
Total loss : 23.996128 Localization Loss : 4.5863276 Confidence Loss :19.409801
Total loss : 24.337835 Localization Loss : 4.638044 Confidence Loss :19.699791
Total loss : 25.32519 Localization Loss : 4.8860497 Confidence Loss :20.43914
Total loss : 24.198246 Localization Loss : 4.8156195 Confidence Loss :19.382626
Total loss : 24.03012 Localization Loss : 4.79578 Confidence Loss :19.23434
Total loss : 24.588648 Localization Loss : 4.7012997 Confidence Loss :19.887348
Iteration0training loss : 28.864874494910243test_loss : 28.908981169104578
Total loss : R(24.152826) Localization Loss : R(4.4638085) Confidence Loss :R(19.689018)
Total loss : R(24.15477) Localization Loss : R

Total loss : R(9.137026) Localization Loss : R(4.452772) Confidence Loss :R(4.684253)
Total loss : R(9.150076) Localization Loss : R(4.3821793) Confidence Loss :R(4.767896)
Total loss : R(9.254977) Localization Loss : R(4.4472103) Confidence Loss :R(4.807767)
Total loss : R(9.134043) Localization Loss : R(4.150708) Confidence Loss :R(4.983334)
Total loss : R(9.554226) Localization Loss : R(4.4896393) Confidence Loss :R(5.064586)
Total loss : R(9.318605) Localization Loss : R(4.3132987) Confidence Loss :R(5.0053062)
Total loss : R(8.994869) Localization Loss : R(4.314078) Confidence Loss :R(4.680791)
Total loss : R(9.45833) Localization Loss : R(4.3687434) Confidence Loss :R(5.089587)
Total loss : R(9.559906) Localization Loss : R(4.3844576) Confidence Loss :R(5.175449)
Total loss : R(9.252845) Localization Loss : R(4.411247) Confidence Loss :R(4.841598)
Total loss : R(9.731741) Localization Loss : R(4.6390734) Confidence Loss :R(5.0926676)
Total loss : R(8.993434) Localization Loss : R

Total loss : R(9.1629095) Localization Loss : R(4.259795) Confidence Loss :R(4.9031143)
Total loss : R(9.774667) Localization Loss : R(4.635238) Confidence Loss :R(5.1394286)
Total loss : R(9.211761) Localization Loss : R(4.2659993) Confidence Loss :R(4.945762)
Total loss : R(9.203905) Localization Loss : R(4.3470106) Confidence Loss :R(4.8568945)
Total loss : R(10.10875) Localization Loss : R(4.754386) Confidence Loss :R(5.3543644)
Total loss : R(9.426613) Localization Loss : R(4.056195) Confidence Loss :R(5.370418)
Total loss : R(10.154618) Localization Loss : R(4.300962) Confidence Loss :R(5.8536563)
Total loss : R(9.024179) Localization Loss : R(4.1888695) Confidence Loss :R(4.8353105)
Total loss : R(9.325409) Localization Loss : R(4.229424) Confidence Loss :R(5.095985)
Total loss : R(9.056131) Localization Loss : R(4.257265) Confidence Loss :R(4.7988667)
Total loss : R(9.938658) Localization Loss : R(4.2456975) Confidence Loss :R(5.6929603)
Total loss : R(9.423528) Localization Lo

Total loss : R(9.725214) Localization Loss : R(4.2754) Confidence Loss :R(5.4498134)
Total loss : R(9.386791) Localization Loss : R(4.5520024) Confidence Loss :R(4.8347893)
Total loss : R(9.569595) Localization Loss : R(4.5908303) Confidence Loss :R(4.978765)
Total loss : R(9.338171) Localization Loss : R(4.103534) Confidence Loss :R(5.234637)
Total loss : R(9.387922) Localization Loss : R(4.14048) Confidence Loss :R(5.2474427)
Total loss : R(9.269301) Localization Loss : R(4.2809563) Confidence Loss :R(4.988345)
Total loss : R(9.673311) Localization Loss : R(4.246294) Confidence Loss :R(5.4270167)
Total loss : R(9.407078) Localization Loss : R(4.5753675) Confidence Loss :R(4.831711)
Total loss : R(9.947153) Localization Loss : R(4.4485273) Confidence Loss :R(5.498626)
Total loss : R(9.074578) Localization Loss : R(4.2606616) Confidence Loss :R(4.8139167)
Total loss : R(8.903511) Localization Loss : R(4.1075177) Confidence Loss :R(4.795994)
Total loss : R(9.147673) Localization Loss : 

Total loss : R(10.016848) Localization Loss : R(4.4589963) Confidence Loss :R(5.5578513)
Total loss : R(9.605906) Localization Loss : R(4.2920375) Confidence Loss :R(5.3138676)
Total loss : R(9.418333) Localization Loss : R(4.5121055) Confidence Loss :R(4.906227)
Total loss : R(9.75676) Localization Loss : R(4.3314533) Confidence Loss :R(5.425306)
Total loss : R(9.479197) Localization Loss : R(4.1487384) Confidence Loss :R(5.3304586)
Total loss : R(10.153254) Localization Loss : R(4.5133605) Confidence Loss :R(5.6398926)
Total loss : R(9.914616) Localization Loss : R(4.5521903) Confidence Loss :R(5.3624253)
Total loss : R(9.700008) Localization Loss : R(4.382461) Confidence Loss :R(5.3175473)
Total loss : R(11.482846) Localization Loss : R(4.737724) Confidence Loss :R(6.7451224)
Total loss : R(8.755403) Localization Loss : R(4.1982927) Confidence Loss :R(4.55711)
Elapsed time 9.531316666666667 min epoch : 3.0

Total loss : R(8.949985) Localization Loss : R(4.185432) Confidence Loss :R(

...

Total loss : R(7.6680226) Localization Loss : R(3.804814) Confidence Loss :R(3.8632085)
Total loss : R(7.2464743) Localization Loss : R(4.1211815) Confidence Loss :R(3.1252925)
Total loss : R(7.90294) Localization Loss : R(4.2080755) Confidence Loss :R(3.694864)
Total loss : R(7.5344305) Localization Loss : R(3.8666723) Confidence Loss :R(3.6677585)
Total loss : R(7.0285416) Localization Loss : R(3.9153805) Confidence Loss :R(3.1131613)
Total loss : R(6.957508) Localization Loss : R(3.5844457) Confidence Loss :R(3.3730621)
Total loss : R(7.6078005) Localization Loss : R(3.8141472) Confidence Loss :R(3.793653)
Total loss : R(7.2306232) Localization Loss : R(3.7526085) Confidence Loss :R(3.4780147)
Total loss : R(7.7631264) Localization Loss : R(4.1263237) Confidence Loss :R(3.6368024)
Total loss : R(7.2045927) Localization Loss : R(3.8468964) Confidence Loss :R(3.3576963)
Total loss : R(7.717119) Localization Loss : R(3.609263) Confidence Loss :R(4.1078563)
Total loss : R(7.6572742) Loc

Total loss : R(6.790854) Localization Loss : R(3.496181) Confidence Loss :R(3.294673)
Total loss : R(7.641488) Localization Loss : R(3.7997642) Confidence Loss :R(3.841724)
Total loss : R(7.2446213) Localization Loss : R(3.9940157) Confidence Loss :R(3.2506058)
Total loss : R(7.235065) Localization Loss : R(3.5423145) Confidence Loss :R(3.6927505)
Total loss : R(7.914678) Localization Loss : R(4.1149917) Confidence Loss :R(3.7996864)
Total loss : R(8.057623) Localization Loss : R(4.050201) Confidence Loss :R(4.0074215)
Total loss : R(7.151471) Localization Loss : R(3.718237) Confidence Loss :R(3.4332342)
Total loss : R(7.551517) Localization Loss : R(3.6436548) Confidence Loss :R(3.9078622)
Total loss : R(7.5904145) Localization Loss : R(3.9493012) Confidence Loss :R(3.6411133)
Total loss : R(7.5918646) Localization Loss : R(3.878492) Confidence Loss :R(3.7133727)
Total loss : R(7.481306) Localization Loss : R(3.5994537) Confidence Loss :R(3.8818526)
Total loss : R(7.278322) Localizati

Total loss : R(7.5235987) Localization Loss : R(3.7848914) Confidence Loss :R(3.7387075)
Total loss : R(7.778046) Localization Loss : R(3.8017719) Confidence Loss :R(3.9762743)
Total loss : R(7.7792187) Localization Loss : R(4.045159) Confidence Loss :R(3.73406)
Total loss : R(7.504535) Localization Loss : R(3.9552345) Confidence Loss :R(3.5493007)
Total loss : R(7.634366) Localization Loss : R(3.899101) Confidence Loss :R(3.7352653)
Total loss : R(7.1295137) Localization Loss : R(3.7447617) Confidence Loss :R(3.384752)
Total loss : R(7.0763206) Localization Loss : R(3.7429981) Confidence Loss :R(3.3333223)
Total loss : R(7.363435) Localization Loss : R(3.5953465) Confidence Loss :R(3.768088)
Total loss : R(6.8614597) Localization Loss : R(3.7054358) Confidence Loss :R(3.1560242)
Total loss : R(7.5603876) Localization Loss : R(3.9130678) Confidence Loss :R(3.6473196)
Total loss : R(7.5545826) Localization Loss : R(4.0347886) Confidence Loss :R(3.5197942)
Total loss : R(7.761732) Locali

Total loss : R(7.7295227) Localization Loss : R(4.0336432) Confidence Loss :R(3.6958795)
Total loss : R(7.2694407) Localization Loss : R(3.7153366) Confidence Loss :R(3.5541043)
Total loss : R(6.992436) Localization Loss : R(3.8199513) Confidence Loss :R(3.1724846)
Total loss : R(7.852188) Localization Loss : R(4.012349) Confidence Loss :R(3.839839)
Total loss : R(7.8551397) Localization Loss : R(3.8422916) Confidence Loss :R(4.012848)
Total loss : R(6.9738274) Localization Loss : R(3.5706892) Confidence Loss :R(3.4031384)
Total loss : R(7.1535034) Localization Loss : R(3.628496) Confidence Loss :R(3.5250077)
Total loss : R(8.165656) Localization Loss : R(4.374501) Confidence Loss :R(3.7911546)
Total loss : R(6.881019) Localization Loss : R(3.5149307) Confidence Loss :R(3.3660884)
Total loss : R(7.5754156) Localization Loss : R(4.000833) Confidence Loss :R(3.5745828)
Total loss : R(7.577733) Localization Loss : R(3.8368385) Confidence Loss :R(3.7408946)
Total loss : R(7.1673307) Locali

Total loss : R(7.161781) Localization Loss : R(3.7208982) Confidence Loss :R(3.4408827)
Total loss : R(6.9480414) Localization Loss : R(3.768834) Confidence Loss :R(3.1792073)
Total loss : R(6.635045) Localization Loss : R(3.5836143) Confidence Loss :R(3.0514305)
Total loss : R(6.971856) Localization Loss : R(3.508736) Confidence Loss :R(3.46312)
Total loss : R(7.174202) Localization Loss : R(3.9606442) Confidence Loss :R(3.2135575)
Total loss : R(7.8649197) Localization Loss : R(4.1141505) Confidence Loss :R(3.7507691)
Total loss : R(7.098902) Localization Loss : R(3.6397808) Confidence Loss :R(3.4591215)
Total loss : R(7.6792765) Localization Loss : R(3.862441) Confidence Loss :R(3.8168356)
Total loss : R(6.9835877) Localization Loss : R(3.697735) Confidence Loss :R(3.2858527)
Total loss : R(7.7577167) Localization Loss : R(3.8810573) Confidence Loss :R(3.8766594)
Total loss : R(7.663266) Localization Loss : R(4.001755) Confidence Loss :R(3.6615112)
Total loss : R(7.493314) Localizat

In [ ]:
#iters_2, trnloss_2, tstloss_2 = mytrain!(ssd300,train_dataset,validation_dataset;learning_rate = 1e-3 )

In [ ]:
#length(validation_dataset)

In [ ]:
import Pkg; Pkg.add("Plots")


In [ ]:
tstloss_c = copy(tstloss)

In [ ]:
trnloss_c = copy(trnloss)

In [ ]:
trnloss_c = trnloss_c ./(length(train_dataset))

In [ ]:
tstloss_c = tstloss_c ./ (length(validation_dataset))

In [ ]:
using Plots; default(fmt=:png,ls=:auto)
plot(iters, trnloss_c, label="train", xlabel="iterations", ylabel="loss")
plot!(iters, tstloss_c, label="test")

In [ ]:
using Plots; default(fmt=:png,ls=:auto)
plot(iters, trnloss, label="train", xlabel="iterations", ylabel="loss")
plot!(iters, tstloss, label="test")

In [ ]:
iteration = 5


In [ ]:
if iteration == 5 || iteration == 10000
    print("yey")
end